In [ ]:
!pip install ipywidgets
!pip install torchaudio
!pip install transformers
!pip install datasets
!pip install librosa

In [ ]:
import numpy as np
import librosa
import os
import random
import re
import json
import pandas as pd
import soundfile as sf
from io import BytesIO
import tempfile
import requests
import scipy.signal as sg
import pydub
import matplotlib.pyplot as plt
from IPython.display import Audio, display

In [ ]:
#import zipfile
#with zipfile.ZipFile("audio.zip","r") as zip_ref:
#    zip_ref.extractall()

In [ ]:
split = 13 #where to split file path to insert new folder toy_data:7 full_data=13
df = pd.read_csv("train_data_full.csv")
csv_name = "train_data_"
df.drop("Unnamed: 0",axis=1, inplace=True)
df.head()

In [ ]:
#Simple Noise Augmentation Test #2: More Noise
#Noise amplification first run: noise = np.asarray(0.01*np.random.randn(len(audio)))
#Noise augmentation second run: noise = np.asarray(0.1*np.random.randn(len(audio)))
#remember to run zip with same settings (folder = augment_0.1)

new_df = df.copy()
run_name = "/full_random_noise_01/"
noise_factor = 0.1

for i in range(len(new_df)):
    audio, sr = librosa.load(df["audio_path"][i])
    audio = librosa.resample(audio,sr, target_sr=16000)
    noise = np.asarray(noise_factor*np.random.randn(len(audio)))
    audio_noise = np.asarray(audio + noise)
    new_filename = df["audio_path"][i][:split]+run_name+df["audio_path"][i][split:]
    sf.write(new_filename,audio_noise,16000)
    new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:split]+run_name+new_df["audio_path"][i][split:]  
    
df = pd.concat([df,new_df],axis=0)
df = df.reset_index(drop=True)
df.to_csv(csv_name+run_name[1:]+".csv")

In [ ]:
df.to_csv("full_random_noise_01.csv")

In [ ]:
#random Gaussian Noise generator
#generate random length noise 

new_df = df.copy()
run_name = "/random_noise_0.1"
noise_factor = 0.1

for i in range(len(new_df)):
    audio, sr = librosa.load(df["audio_path"][i])
    audio = librosa.resample(audio,sr, target_sr=16000)
    
    noise_strength = noise_factor
    
    noise = np.random.uniform(0.0,noise_strength,len(audio))

    random_number_chunks = 10 #np.random.randint(1,10)

    chunk = int(len(audio)/random_number_chunks)
    
    mask=[]
    
    for j in range(1,random_number_chunks+1): 

        masking_range = noise[(j*chunk-chunk):(j*chunk)]
        start = np.random.randint(0,len(masking_range))
        stop = start + int(len(masking_range))
        masking_range[start:stop] = 0
    
    audio_noise = np.asarray(audio + noise)
    
    new_filename = df["audio_path"][i][:split]+run_name+df["audio_path"][i][split:]
    
    sf.write(new_filename,audio_noise,16000)

    new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:split]+run_name+new_df["audio_path"][i][split:]  
    
df = pd.concat([df,new_df],axis=0)
df = df.reset_index(drop=True)
df.to_csv(csv_name+run_name[1:]+".csv")


In [ ]:
#every 10 ms there is a chance that audio will be masked --> Chopy audio

new_df = df.copy()
run_name = "/random_augment"
mask_ms = 10

for i in range(len(new_df)):
    
    masked_audio, sr = librosa.load(df["audio_path"][i])
    masked_audio = librosa.resample(masked_audio,sr, target_sr=16000)
    index_length = int(sr/mask_ms)
    
    for j in range(0,len(masked_audio),index_length):
        random_chance = np.random.randint(1,10)
        if random_chance > 7:
            masked_audio[j:j+index_length] = 0
    
    new_filename = df["audio_path"][i][:split]+run_name+df["audio_path"][i][split:]
    
    sf.write(new_filename,masked_audio,16000)

    new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:split]+run_name+new_df["audio_path"][i][split:]  
    
df = pd.concat([df,new_df],axis=0)
df = df.reset_index(drop=True)
df.to_csv(csv_name+run_name[1:]+".csv")


In [ ]:
#speed change slow

new_df = df.copy()
run_name = "/slow_10pct"
speed_change = 1.1 #change < 1 is faster change > 1 is slower

for i in range(len(new_df)):
    
    masked_audio, sr = librosa.load(df["audio_path"][i])
    masked_audio = librosa.resample(masked_audio,sr, target_sr=16000*speed_change)
    
    new_filename = df["audio_path"][i][:split]+run_name+df["audio_path"][i][split:]
    
    sf.write(new_filename,masked_audio,16000)

    new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:split]+run_name+new_df["audio_path"][i][split:]  
    
df = pd.concat([df,new_df],axis=0)
df = df.reset_index(drop=True)
df.to_csv(csv_name+run_name[1:]+".csv")

In [ ]:
#speed change fast

new_df = df.copy()
run_name = "/fast_10pct"
speed_change = 0.9 #change < 1 is faster change > 1 is slower

for i in range(len(new_df)):
    
    masked_audio, sr = librosa.load(df["audio_path"][i])
    masked_audio = librosa.resample(masked_audio,sr, target_sr=16000*speed_change)
    
    new_filename = df["audio_path"][i][:split]+run_name+df["audio_path"][i][split:]
    
    sf.write(new_filename,masked_audio,16000)

    new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:split]+run_name+new_df["audio_path"][i][split:]  
    
df = pd.concat([df,new_df],axis=0)
df = df.reset_index(drop=True)
df.to_csv(csv_name+run_name[1:]+".csv")

In [ ]:
#random high pass filter


new_df = df.copy()
run_name = "/random_high_pass"


for i in range(len(new_df)):
    
        audio,sr = librosa.load(df["audio_path"][i])
        
        audio = librosa.resample(audio,sr, target_sr=16000)
        
        high_cutoff = random.randint(200,1000)

        b, a = sg.butter(4, high_cutoff / (sr / 2.), 'high')
        
        cutoff_audio = sg.filtfilt(b, a, audio)
        
        new_filename = df["audio_path"][i][:split]+run_name+df["audio_path"][i][split:]

        sf.write(new_filename,cutoff_audio,16000)

        new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:split]+run_name+new_df["audio_path"][i][split:]
        
        
df = pd.concat([df,new_df],axis=0)
df = df.reset_index(drop=True)
df.to_csv(csv_name+run_name[1:]+".csv")
        
    
    

In [ ]:
#random low pass filter

new_df = df.copy()
run_name = "/random_low_pass"


for i in range(len(new_df)):
    
        audio,sr = librosa.load(df["audio_path"][i])
        
        audio = librosa.resample(audio,sr, target_sr=16000)
        
        high_cutoff = random.randint(1000,1500)

        b, a = sg.butter(4, high_cutoff / (sr / 2.), 'high')
        
        cutoff_audio = sg.filtfilt(b, a, audio)
        
        new_filename = df["audio_path"][i][:split]+run_name+df["audio_path"][i][split:]

        sf.write(new_filename,cutoff_audio,16000)

        new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:split]+run_name+new_df["audio_path"][i][split:]
        
        
df = pd.concat([df,new_df],axis=0)
df = df.reset_index(drop=True)
df.to_csv(csv_name+run_name[1:]+".csv")
        
    
    

In [ ]:
#augmenting testset
#number of augmentations availible: 7
### This whole block could be solved more elegantly by defining the funktionality above as seperate functions
### TODO: implement seperate augemntations as funkitons

df.drop("Unnamed: 0",axis=1, inplace=True)
new_df = df.copy()
run_name = "full_test_random_augment/"
speed_change_slow = 1.11
speed_change_fast = 0.913
noise_factor = 0.012
mask_ms=27

for i in range(len(new_df)):
    
    pick = random.randint(1,7)
    
    if pick == 1: #simple random gaussian noise
        df.loc[i,"augmentation"] = "random noise"
        
        noise_audio, sr = librosa.load(df["audio_path"][i])

        noise_audio = librosa.resample(noise_audio,sr, target_sr=16000)
       
        noise = np.asarray(noise_factor*np.random.randn(len(noise_audio)))
        
        audio_noise = np.asarray(noise_audio + noise)
        
        new_filename = df["audio_path"][i][:13]+run_name+df["audio_path"][i][13:]
        
        sf.write(new_filename,audio_noise,16000)
        
        new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:13]+run_name+new_df["audio_path"][i][13:]
        
    elif pick == 2: #noise bursts
         
        df.loc[i,"augmentation"] = "noise burst"
        
        random_audio, sr = librosa.load(df["audio_path"][i])
        
        random_audio = librosa.resample(random_audio,sr, target_sr=16000)


        random_noise = np.random.uniform(0.0,noise_factor*10,len(random_audio))

        random_number_chunks = 10 #np.random.randint(1,10)

        chunk = int(len(random_audio)/random_number_chunks)

        for j in range(1,random_number_chunks+1): 

            masking_range = random_noise[(j*chunk-chunk):(j*chunk)]
            
            start = np.random.randint(0,len(masking_range))
            stop = start + int(len(masking_range))
            
            masking_range[start:stop] = 0
                        
        audio_noise = np.asarray(random_audio + random_noise)

        new_filename = df["audio_path"][i][:13]+run_name+df["audio_path"][i][13:]

        sf.write(new_filename,audio_noise,16000)
        
        new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:13]+run_name+new_df["audio_path"][i][13:]


    if pick == 3: #mask out parts of audio
        
        df.loc[i,"augmentation"] = "masked audio"
        
        masked_audio, sr = librosa.load(df["audio_path"][i])
        masked_audio = librosa.resample(masked_audio,sr, target_sr=16000)
        index_length = int(sr/mask_ms)

        for j in range(0,len(masked_audio),index_length):
            random_chance = np.random.randint(1,10)
            if random_chance > 7:
                masked_audio[j:j+index_length] = 0

        new_filename = df["audio_path"][i][:13]+run_name+df["audio_path"][i][13:]

        sf.write(new_filename,masked_audio,16000)

        new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:13]+run_name+new_df["audio_path"][i][13:] 
    
    if pick == 4: #slow down audio
        df.loc[i,"augmentation"] = "slow"
        slow_audio, sr = librosa.load(df["audio_path"][i])
        slow_audio = librosa.resample(slow_audio,sr, target_sr=16000*speed_change_slow)

        new_filename = df["audio_path"][i][:13]+run_name+df["audio_path"][i][13:]

        sf.write(new_filename,slow_audio,16000)

        new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:13]+run_name+new_df["audio_path"][i][13:]
        
    if pick == 5: #speed up audio
        df.loc[i,"augmentation"] = "fast"
        fast_audio, sr = librosa.load(df["audio_path"][i])
        fast_audio = librosa.resample(fast_audio,sr, target_sr=16000*speed_change_fast)

        new_filename = df["audio_path"][i][:13]+run_name+df["audio_path"][i][13:]

        sf.write(new_filename,fast_audio,16000)

        new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:13]+run_name+new_df["audio_path"][i][13:] 
        
    if pick == 6: #random highpass
        df.loc[i,"augmentation"] = "high_pass"
        high_cutoff_audio,sr = librosa.load(df["audio_path"][i])
        
        high_cutoff_audio = librosa.resample(high_cutoff_audio,sr, target_sr=16000)
        
        high_cutoff = random.randint(200,1000)

        b, a = sg.butter(4, high_cutoff / (sr / 2.), 'high')
        
        cutoff_audio = sg.filtfilt(b, a, high_cutoff_audio)
        
        new_filename = df["audio_path"][i][:13]+run_name+df["audio_path"][i][13:]

        sf.write(new_filename,cutoff_audio,16000)

        new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:13]+run_name+new_df["audio_path"][i][13:]
        
    if pick == 7: #random low pass
        df.loc[i,"augmentation"] = "low pass"
        low_cutoff_audio,sr = librosa.load(df["audio_path"][i])
        
        low_cutoff_audio = librosa.resample(low_cutoff_audio,sr, target_sr=16000)
        
        low_cutoff = random.randint(1000,1500)

        b, a = sg.butter(4, low_cutoff / (sr / 2.), 'low')
        
        low_cutoff_audio = sg.filtfilt(b, a, low_cutoff_audio)
        
        new_filename = df["audio_path"][i][:13]+run_name+df["audio_path"][i][13:]

        sf.write(new_filename,low_cutoff_audio,16000)

        new_df.loc[i,"audio_path"] = new_df["audio_path"][i][:13]+run_name+new_df["audio_path"][i][13:]
    
    
     
    
       
df = pd.concat([df,new_df],axis=0)
df = df.reset_index(drop=True)
df.to_csv("full_data_augment_random.csv")    